<a href="https://colab.research.google.com/github/Tanishta15/SSDigimark/blob/main/review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from textblob import TextBlob
import http.client
import json

BUSINESS_NAME = "GOOGLE"
LOCATION = "California, U.S"

# Your RapidAPI credentials (fill your actual key and host)
RAPIDAPI_KEY = "01cd200125msha6a048803311025p15b3edjsn5ffb9e658b8d"
RAPIDAPI_HOST = "local-business-data.p.rapidapi.com"

In [ ]:
def get_reviews(business_name, location):
    # Step 1: Search business to get business_id
    url = "https://local-business-data.p.rapidapi.com/search"
    querystring = {
        "query": f"{business_name} {location}",
        "limit": "1",
        "language": "en"
    }
    headers = {
        "X-RapidAPI-Key": RAPIDAPI_KEY,
        "X-RapidAPI-Host": RAPIDAPI_HOST
    }

    print("Starting business review fetch...")
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    print(f"Search Query Sent: {business_name} {location}")
    print("Search API Raw Response:", data)

    if "data" not in data or not data["data"]:
        print("No business found.")
        return []

    # Extract business_id (from your example: "business_id" key)
    business_id = data["data"][0].get("business_id")
    if not business_id:
        print("Business ID not found in search results.")
        return []

    # Step 2: Use http.client to call /review-details with business_id
    conn = http.client.HTTPSConnection(RAPIDAPI_HOST)

    headers_http = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': RAPIDAPI_HOST
    }

    # Build the request path for review-details endpoint
    request_path = f"/review-details?business_id={business_id}&region=us&language=en"

    conn.request("GET", request_path, headers=headers_http)
    res = conn.getresponse()
    raw_data = res.read()
    json_data = json.loads(raw_data.decode("utf-8"))

    # Extract reviews safely
    reviews = json_data.get("reviews", [])

    if not reviews:
        print("No reviews found.")
    else:
        print(f"Found {len(reviews)} reviews.")

    return reviews

In [ ]:
def analyze_sentiment(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity

    if polarity > 0.2:
        return "positive"
    elif polarity < -0.2:
        return "negative"
    else:
        return "neutral"

In [ ]:
def generate_response(sentiment):
    responses = {
        "positive": "Thank you for your kind words! We're glad you had a great experience.",
        "neutral": "Thanks for your feedback. We're always working to improve.",
        "negative": "We're sorry to hear that. Please reach out to us so we can make it right."
    }
    return responses.get(sentiment, "")

In [ ]:
def main():
    reviews = get_reviews(BUSINESS_NAME, LOCATION)

    if not reviews:
        print("No reviews available to analyze.")
        return

    print(f"\n--- Reviews for {BUSINESS_NAME} ---\n")
    for review in reviews:
        text = review.get("text", "")
        if not text.strip():
            continue
        sentiment = analyze_sentiment(text)
        response = generate_response(sentiment)

        print(f"📝 Review: {text}")
        print(f"🔍 Sentiment: {sentiment}")
        print(f"🤖 Auto-response: {response}")
        print("-" * 50)


if __name__ == "__main__":
    main()

Starting business review fetch...
Search Query Sent: GOOGLE California, U.S
Search API Raw Response: {'status': 'OK', 'request_id': 'b85016af-42b5-4ccd-8c42-a5ddd05dd8db', 'parameters': {'query': 'GOOGLE California, U.S', 'language': 'en', 'region': 'us', 'lat': 37.09024, 'lng': -95.712891, 'zoom': 13, 'limit': 1, 'offset': 0, 'extract_emails_and_contacts': False}, 'data': [{'business_id': '0x808fba02425dad8f:0x6c296c66619367e0', 'google_id': '0x808fba02425dad8f:0x6c296c66619367e0', 'place_id': 'ChIJj61dQgK6j4AR4GeTYWZsKWw', 'google_mid': '/m/03bby1', 'phone_number': '+16502530000', 'name': 'Googleplex', 'latitude': 37.4220541, 'longitude': -122.08532419999999, 'full_address': 'Googleplex, 1600 Amphitheatre Pkwy, Mountain View, CA 94043', 'review_count': 9614, 'rating': 4.2, 'timezone': 'America/Los_Angeles', 'opening_status': 'Open ⋅ Closes 5\u202fPM', 'working_hours': {'Wednesday': ['8 AM–5 PM'], 'Thursday': ['8 AM–5 PM'], 'Friday': ['8 AM–5 PM'], 'Saturday': ['Closed'], 'Sunday': ['